In [1]:
import os
os.chdir("../")

In [3]:
import dagshub

dagshub.init(repo_owner='snigdhrf', repo_name='End-to-End-ML-Pipeline-with-MLFlow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/opt/anaconda3/envs/images_classifier/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=990a3f4e-1c20-4ec4-ae1c-3041454dd208&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=81f5e12d5e5f128d51e21d91b0a1bb3cc0c2808da6b800a0908ae69e1cbc1387




Accessing as snigdhrf

Initialized MLflow to track repo "snigdhrf/End-to-End-ML-Pipeline-with-MLFlow-DVC"

Repository snigdhrf/End-to-End-ML-Pipeline-with-MLFlow-DVC initialized!

In [2]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingEvaluationConfig:
    # Training-specific
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path

    # Shared
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

    # Evaluation-specific
    path_of_model: Path
    all_params: dict
    mlflow_uri: str

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_evaluation_config(self) -> TrainingEvaluationConfig:
        training_config = self.config.training
        evaluation_config = self.config.evaluation
        prepare_base_model = self.config.prepare_base_model
        training_data = self.config.data_ingestion.unzip_dir

        create_directories([
                Path(training_config.root_dir)
            ])

        return TrainingEvaluationConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),

            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_is_augmentation=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE,

            path_of_model=Path(evaluation_config.path_of_model),
            all_params=self.params,
            mlflow_uri=evaluation_config.mlflow_uri
    )

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [21]:
class Evaluation:
    def __init__(self, config: TrainingEvaluationConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")




In [22]:
try:
    config = ConfigurationManager()
    merged_config = config.get_training_evaluation_config()

    model_pipeline = Evaluation(config=merged_config)
    
    model_pipeline.get_base_model()
    model_pipeline.train_valid_generator()
    model_pipeline.train()
    model_pipeline.evaluation()
    model_pipeline.log_into_mlflow()

except Exception as e:
    raise e

[2025-06-16 14:58:54,041: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-16 14:58:54,044: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-16 14:58:54,045: INFO: common: created directory at: artifacts]
[2025-06-16 14:58:54,046: INFO: common: created directory at: artifacts/training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
5/5 [==============================] - 1s 121ms/step - loss: 0.0759 - accuracy: 0.9706
[2025-06-16 14:58:58,889: INFO: common: json file saved at: scores.json]
[2025-06-16 14:59:16,984: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /snigdhrf/End-to-End-ML-Pipeline-with-MLFlow-DVC.mlflow/api/2.0/mlflow/runs/create]


2025/06/16 14:59:18 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-06-16 14:59:18,686: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/xt/vrrq87pn7p1gnmx8xk0tqs080000gn/T/tmp8rcd0h3l/model/data/model/assets
[2025-06-16 14:59:18,954: INFO: builder_impl: Assets written to: /var/folders/xt/vrrq87pn7p1gnmx8xk0tqs080000gn/T/tmp8rcd0h3l/model/data/model/assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/06/16 14:59:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
